## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [ ]:
import os
import cv2
import glob
import numpy as np
import glob
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
import matplotlib.pyplot as plt

# calibrate camera
def calibrate_camera(calibration_files="./camera_cal/calibration*.jpg"):
    nx = 9 #enter the number of inside corners in x
    ny = 6 #enter the number of inside corners in y

    objpoints = [] #3D object points
    imgpoints = [] #2D image points

    # prepare object points
    objp = np.zeros((nx * ny, 3), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2)

    images = glob.glob(calibration_files)

    for idx, fname in enumerate(images):
        # read image, BGR format in cv2
        img = cv2.imread(fname)  
        # Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

        # If found, add to the list of imgpoints and objpoints
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    return cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# undistort image
def undistort_image(img, mtx, dist):
    return cv2.undistort(img, mtx, dist, None, mtx)

def rgb_select(image, ch=0, thresh=(0, 255)):
    channel = image[:,:,ch]
    binary = np.zeros_like(channel)
    binary[(channel > thresh[0]) & (channel <= thresh[1])] = 1
    return binary

def hls_select(image, ch=2, thresh=(0, 255)):
    hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    channel = hls[:,:,ch]
    binary = np.zeros_like(channel)
    binary[(channel > thresh[0]) & (channel <= thresh[1])] = 1
    return binary

def hsv_select(image, ch=1, thresh=(0, 255)):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    channel = hsv[:,:,ch]
    binary = np.zeros_like(channel)
    binary[(channel > thresh[0]) & (channel <= thresh[1])] = 1
    return binary

def hsv_yellow_mask(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    yellow_hsv_low  = np.array([ 15, 100, 100])
    yellow_hsv_high = np.array([ 35, 255, 255])
    yellow_mask = cv2.inRange(hsv, yellow_hsv_low, yellow_hsv_high)
    return yellow_mask

def hsv_white_mask(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    white_hsv_low  = np.array([ 0,   0,   205])
    white_hsv_high = np.array([ 255,  30, 255])
    white_mask = cv2.inRange(hsv, white_hsv_low, white_hsv_high)
    return white_mask

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

def abs_sobel_thresh_gray(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = np.copy(img)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
        
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
        
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobelx = np.absolute(sobelx)
    abs_sobely = np.absolute(sobely)
    absgraddir = np.arctan2(abs_sobely, abs_sobelx)
    binary_output = np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return binary_output

# color gradient threshold
def color_gradient_threshold(image):
    # Choose a Sobel kernel size
    ksize = 31 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(51, 255))
    hls_s_binary = hls_select(image, ch=2, thresh=(100, 255))
    hls_v_binary = hls_select(image, ch=1, thresh=(120, 160))
    rgb_r_binary = rgb_select(image, ch=0, thresh=(220, 255))
    yellow_mask = hsv_yellow_mask(image)
    white_mask = hsv_white_mask(image)
    color_mask = cv2.bitwise_or(yellow_mask, white_mask)

    #hls_s_binary1 = hls_select(image, ch=2, thresh=(75, 165))
    #hls_v_binary1 = hls_select(image, ch=1, thresh=(75, 165))
    #grade_s_x1 = abs_sobel_thresh_gray(hls_s_binary1, orient='x', sobel_kernel=ksize, thresh=(50, 200))
    #grade_s_xy1 = abs_sobel_thresh_gray(grade_s_x1, orient='y', sobel_kernel=ksize, thresh=(50, 200))
    #grade_v_x1 = abs_sobel_thresh_gray(hls_v_binary1, orient='x', sobel_kernel=ksize, thresh=(50, 200))
    #grade_v_xy1 = abs_sobel_thresh_gray(grade_v_x1, orient='y', sobel_kernel=ksize, thresh=(50, 200))

    combined = np.zeros_like(gradx)
    #combined[((gradx == 1) | (rgb_r_binary == 1) | ((hls_s_binary == 1) & (hls_v_binary == 1)) | (yellow_mask == 1))] = 1
    #combined[((color_mask >= 255) | ((hls_s_binary == 1) & (hls_v_binary == 1)))] = 1
    #combined[((color_mask >= 255) | ((grade_s_xy1 == 1) & (grade_v_xy1 == 1)))] = 1
    combined[((gradx == 1) | (color_mask >= 255) | ((hls_s_binary == 1) & (hls_v_binary == 1)))] = 1
    return combined

# unwarp image
def unwarp_image(img, src, dst):
    img_height, img_width = img.shape[:2]
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, (img_width, img_height), flags=cv2.INTER_LINEAR)
    return warped, M, Minv

# check if the left/right fits are parallel within a given threshold
def is_parallel(left_fit, right_fit, threshold = (0.01, 0.5)):
    first_coefficient_diff = np.abs(left_fit[0] - right_fit[0])
    second_coefficient_diff = np.abs(left_fit[1] - right_fit[1])
    print("first coefficient diff : {} second coefficient diff : {}".format(first_coefficient_diff, second_coefficient_diff))
    return first_coefficient_diff < threshold[0] and second_coefficient_diff < threshold[1]

# get distance between fits
def is_distance_in_range(first_fitx, second_fitx, threshold = (600,900)):
    dist = np.abs(first_fitx[-1] - second_fitx[-1])
    xm_per_pix = 3.7/700 # To be cleaned up
    print("fit pos : {:.2f} px {:.2f} px".format(first_fitx[-1], second_fitx[-1]))
    print("fit distance : {:.2f} px {:.2f} m".format(dist, dist * xm_per_pix))
    return dist > threshold[0] and dist < threshold[1]

# removal of outlier samples
def outlier_removal(x, y, q=10):
    x = np.array(x)
    y = np.array(y)

    lower_bound = np.percentile(x, q)
    upper_bound = np.percentile(x, 100 - q)
    selection = (x >= lower_bound) & (x <= upper_bound)
    return x[selection], y[selection]

# sliding window implementation
# for a given image, returns a list of points on the left/right lines
def sliding_windows(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # initialize sliding windows
    sliding_windows_list = []
    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # save the sliding window
        left_window = [(win_xleft_low, win_y_low), (win_xleft_high, win_y_high)]
        right_window = [(win_xright_low, win_y_low), (win_xright_high, win_y_high)]
        sliding_windows_list.append((left_window, right_window))
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
    # Return
    return sliding_windows_list, leftx, lefty, rightx, righty

# from existing polyfit
def find_lane_lines_from_existing(binary_warped, left_fit, right_fit):
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def sliding_windows_image(binary_warped, sliding_windows_list, color=(0,255,0), border_thickness=3):
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    for window in sliding_windows_list:
        win_left = window[0]
        win_right = window[1]
        # Draw the windows on the visualization image
        cv2.rectangle(out_img, win_left[0], win_left[1], color, border_thickness)
        cv2.rectangle(out_img, win_right[0], win_right[1], color, border_thickness)
    return out_img

def overlay_image(undist, binary_warped, Minv, left_fit, right_fit):

    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    y_eval = np.max(ploty)
    
    img_center = binary_warped.shape[1] / 2
    lane_center = (left_fit[-1] + right_fit[-1]) / 2
    diff = (lane_center - img_center) * xm_per_pix
    left_or_right = 'left' if diff < 0 else 'right'

    center_fit = (left_fit + right_fit) / 2
    center_curverad = ((1 + (2 * center_fit[0] * y_eval * ym_per_pix + center_fit[1])**2)**1.5) / np.absolute(2 * center_fit[0])

    # Adding Curvature and Distance information
    result = cv2.putText(
        result,
        'Curvature : {:.2f} m'.format(center_curverad),(50,50), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1,
        (255,255,255),
        2,
        cv2.LINE_AA)
    result = cv2.putText(
        result,
        'Distance : {:.2f} m {} of the center'.format(abs(diff), left_or_right),
        (50,100), 
        cv2.FONT_HERSHEY_SIMPLEX, 
        1,
        (255,255,255),
        2,
        cv2.LINE_AA)
    return result

# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self, moving_average=4):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        #polynomial coefficients for the most recent fit
        self.current_fit = []
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        #moving average, number of samples to keep
        self.moving_average = moving_average
        
    def add(self, fit):
        if fit is not None:
            self.detected = True
            self.current_fit.append(fit)
            # keep the recent
            if len(self.current_fit) > self.moving_average:
                self.current_fit = self.current_fit[len(self.current_fit) - self.moving_average:]
            self.best_fit = np.average(self.current_fit, axis=0)
        else:
            self.detected = False

# return a random prefix with numbers and alphabets of N length, used as a session id for output files
def random_prefix(N=5):
    import random, string
    return ''.join(random.SystemRandom().choice(string.ascii_uppercase + string.digits) for _ in range(N))

# project class
class AdvancedLaneLine:
    def __init__(self, session=random_prefix(), debug=False):
        # calibrate camera
        self.ret, self.mtx, self.dist, self.rvecs, self.tvecs = calibrate_camera()

        # previous lines
        self.left_line = Line()
        self.right_line = Line()
        
        # perspective transform coordinates
        bottom_left = (190, 720)
        bottom_right = (1130, 720)
        top_left = (585, 455)
        top_right = (705, 455)

        # source points
        self.src = np.float32([top_right,
                  bottom_right, 
                  bottom_left, 
                  top_left])

        # desired destination
        dst_top_left = (200, 50)
        dst_top_right = (1000, 50)
        dst_bottom_right = (1000, 720)
        dst_bottom_left = (200, 720)

        self.dst = np.float32([dst_top_right,
                  dst_bottom_right, 
                  dst_bottom_left, 
                  dst_top_left])
        
        self.frame_number = 0
        self.session = session
        self.debug = debug
        return

    # get the session/frame/file name for the given file name suffix
    def get_session_frame_file_name(self, file_suffix):
        return "{}_{}_{}".format(self.session, self.frame_number, file_suffix)

    # save intermediate images, debug purposes 
    def save_image(self, img, suffix="out", out_folder="output"):
        if self.debug == True:
            outfile = "./{}/{}.jpg".format(out_folder, self.get_session_frame_file_name(suffix))
            mpimg.imsave(outfile, img)
        return

    def find_lane_lines(self, unwarp):
        if self.left_line.detected == False or self.right_line.detected == False:
            print("Sliding windows ...")
            sliding_windows_list, leftx, lefty, rightx, righty = sliding_windows(unwarp)
            slide_win_img = sliding_windows_image(unwarp, sliding_windows_list)
            self.save_image(slide_win_img, "slide")
        else:
            print("From existing ...")
            leftx, lefty, rightx, righty = find_lane_lines_from_existing(unwarp, self.left_line.best_fit, self.right_line.best_fit)

        left_fit, right_fit = (None, None)
        # remove outliers
        if len(leftx) > 0:
            leftx, lefty = outlier_removal(leftx, lefty)
            if len(leftx) > 2:
                left_fit = np.polyfit(lefty, leftx, 2)
        if len(rightx) > 0:
            rightx, righty = outlier_removal(rightx, righty)
            if len(rightx) > 2:
                right_fit = np.polyfit(righty, rightx, 2)
        if left_fit is not None or right_fit is not None:
            ploty = np.linspace(0, unwarp.shape[0]-1, unwarp.shape[0])
            left_fitx, right_fitx = (None, None)
            if left_fit is not None:
                left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
            if right_fit is not None:
                right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        if left_fitx is not None and right_fitx is not None:
            # Create an output image to draw on and  visualize the result
            lines_out = np.dstack((unwarp, unwarp, unwarp))*255
            ploty = np.linspace(0, unwarp.shape[0]-1, unwarp.shape[0])

            # Recast the x and y points into usable format for cv2.fillPoly()
            pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
            pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
            pts = np.hstack((pts_left, pts_right))
            # Draw the lane onto the warped blank image
            cv2.polylines(lines_out, np.int32([pts]), 1, (255,255,0))
            self.save_image(lines_out, "lines")
            
        # both fits are not None
        if left_fit is not None and right_fit is not None:
            # check if the fits are looking valid
            if is_parallel(left_fit, right_fit) and is_distance_in_range(left_fitx, right_fitx):
                self.left_line.add(left_fit)
                self.right_line.add(right_fit)
                print("All good for frame : {}".format(self.frame_number))
                return
            else:
                print("Skipping for frame : {}".format(self.frame_number))
        else:
            if left_fit is None:
                print("left_fit is None for frame : {}".format(self.frame_number))
            if right_fit is None:
                print("right_fit is None for frame : {}".format(self.frame_number))
        # reset the flags so that next time, it will do sliding windows
        self.left_line.detected = False
        self.right_line.detected = False
        return
    
    def overlay_lane_lines(self, undist, unwarp, Minv):
        out = overlay_image(undist, unwarp, Minv, self.left_line.best_fit, self.right_line.best_fit)
        return out
    
    def image_pipeline(self, image):
        self.save_image(image, "org")
        # 1. undistort image
        undist = undistort_image(image, self.mtx, self.dist)
        self.save_image(undist, "undist")
        # 2. color and gradient threshold
        threshold = color_gradient_threshold(undist)
        self.save_image(threshold,"thres")
        # 3. perspective transform
        unwarp, M, Minv = unwarp_image(threshold, self.src, self.dst)
        self.save_image(unwarp, "unwarp")
        # 4. find lane lines
        self.find_lane_lines(unwarp)
        # 5. overlay lane lines on image
        out_img = self.overlay_lane_lines(undist, unwarp, Minv)
        self.save_image(out_img, "out")
        # increment the frame counter
        self.frame_number += 1
        return out_img

# try against all test images
images = glob.glob('./test_images/*.jpg')

for idx, fname in enumerate(images):
    print("processing image ({}) ...".format(fname))
    # instance of the project class
    file_path_split = os.path.splitext(os.path.basename(fname))
    file_name = file_path_split[-2]
    advanced_lane_line = AdvancedLaneLine(session=file_name, debug=True)
    img = mpimg.imread(fname)
    out = advanced_lane_line.image_pipeline(img)

# create an instance per video
advanced_lane_line_1 = AdvancedLaneLine(session="session1", debug=False)

print("processing project video ...")
# project video
input_video = VideoFileClip("./project_video.mp4")
outclip = input_video.fl_image(advanced_lane_line_1.image_pipeline)
outclip.write_videofile("./output/project_video.mp4", audio=False)

'''
# create an instance per video
advanced_lane_line_2 = AdvancedLaneLine(session="session2", debug=True)
input_video = VideoFileClip("./challenge_video.mp4")
outclip = input_video.fl_image(advanced_lane_line_2.image_pipeline)
outclip.write_videofile("./output/challenge_video.mp4", audio=False)
# challenge video #1
# advanced_lane_line_3 = AdvancedLaneLine(session="session3", debug=False)
# challenge video #2
'''